Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [1]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [2]:
def print_transit(query_json):
    for a in query_json['plan']['itineraries']:
        print('start: '+ tclean(a['startTime']))
        print('end: '+ tclean(a['endTime']))
        print('duration: '+ str(a['duration']/60) + ' minutes')
        print('waitingTime: '+ str(a['waitingTime']/60) + ' minutes')
        print('transitTime: '+ str(a['transitTime']/60) + ' minutes')
        print('walkDistance: '+ str(a['walkDistance']/1000*.621371) + ' miles')
        print('transfers: '+ str(a['transfers']))
        for each in a['legs']:
            print('   StartTime: '+ tclean(each['startTime']) + " mode: " + each['mode'])
            print("   headsign: " + str(each.get('headsign')))
            print('   endTime:   ' + tclean(each['endTime']) + " duration: " + str(each['duration']/60))    

In [3]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')

df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [4]:
def tclean(dto):
    return str(datetime.fromtimestamp(dto/1000))

In [5]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose
2,37.400787,-121.868619,1684 Garvey Pl,37.400281,-121.939691,3331 N. 1st St VTA
3,37.400281,-121.939691,3331 N. 1st St VTA,37.333539,-121.885089,"116 Paseo De San Antonio Walk, San Jose, CA 95..."


In [11]:
row = 0
payload = {}
payload.update({'fromPlace': '%s,%s' % (df['start_lat'][row],df['start_long'][row])})
payload.update({'toPlace': '%s, %s' % (df['dest_lat'][row], + df['dest_long'][row])})
payload.update({'mode':'WALK,BUSISH,TRAINISH'})
payload.update({'date':'2016-07-13'})
payload.update({'time':'09:30'})
payload.update({'maxTransfers':3,'optimize':'QUICK'})
payload.update({'numItineraries':5})
payload.update({'maxWalkDistance':5000})
payload.update({'bikeSpeed':10})
payload.update({'arriveBy':'false'})
#payload.update ({'date': '2016-07-13', 'mode': ['WALK','BUSISH','TRAINISH']})

In [12]:
r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
#r.text
print_transit(r.json())

start: 2016-07-13 09:32:05
end: 2016-07-13 10:51:51
duration: 79.76666666666667 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 15.0 minutes
walkDistance: 3.1286825391695934 miles
transfers: 0
   StartTime: 2016-07-13 09:32:05 mode: WALK
   headsign: None
   endTime:   2016-07-13 09:33:59 duration: 1.9
   StartTime: 2016-07-13 09:34:00 mode: BUS
   headsign: 70 GREAT MALL
   endTime:   2016-07-13 09:49:00 duration: 15.0
   StartTime: 2016-07-13 09:49:01 mode: WALK
   headsign: None
   endTime:   2016-07-13 10:51:51 duration: 62.833333333333336
start: 2016-07-13 09:46:05
end: 2016-07-13 11:05:51
duration: 79.76666666666667 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 15.0 minutes
walkDistance: 3.1286825391695934 miles
transfers: 0
   StartTime: 2016-07-13 09:46:05 mode: WALK
   headsign: None
   endTime:   2016-07-13 09:47:59 duration: 1.9
   StartTime: 2016-07-13 09:48:00 mode: BUS
   headsign: 70 GREAT MALL
   endTime:   2016-07-13 10:03:00 duration: 15.

In [ ]:
, '.join([str(x) for x in list])
''.join(map(str,list))